In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

In [ ]:
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory
print("✅ ADK components imported successfully.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
from google.adk.runners import Runner
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

In [ ]:
import json
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService

APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session
MODEL_NAME = "gemini-2.5-flash-lite"



def find_scholarships(profile: dict, dataset_path: str, top_k: int = 5):
    import json

    try:
        with open(dataset_path, "r") as f:
            scholarships = json.load(f)
    except Exception as e:
        return {"status": "error", "error_message": f"Failed to load dataset: {str(e)}"}

    eligible = []

    funding_pref = str(profile.get("funding", "")).lower().replace("$", "").replace(",", "")

    for s in scholarships:
        degree_ok = (
            str(profile.get("degree", "")).lower() == "any"
            or profile.get("degree") in s.get("degrees", "")
        )

        country_ok = (
            str(profile.get("country", "")).lower() == "any"
            or profile.get("country", "").lower() in s.get("location", "").lower()
        )

        if degree_ok and country_ok:
            score = 0

            scholarship_funds_str = str(s.get("funds", "")).lower().replace("$", "").replace(",", "")

            if funding_pref == "any" or funding_pref in scholarship_funds_str:
                score += 1

            s["score"] = score
            eligible.append(s)

    if not eligible:
        return {"status": "success", "scholarships": [], "message": "No matching scholarships found"}

    return {
        "status": "success",
        "scholarships": sorted(eligible, key=lambda x: x["score"], reverse=True)[:top_k]
    }

        
          
# Define scope levels for state keys (following best practices)
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# This demonstrates how tools can write to session state using tool_context.
# The 'user:' prefix indicates this is user-specific data.
def save_userinfo(
    tool_context: ToolContext, **kwargs
) -> dict[str, any]:
    """
    Flexible tool to save ANY user information into session state.
    All keys are auto-prefixed with 'user:'.

    Example usage:
    save_userinfo(name="Imon", country="Bangladesh", degree="BSc CSE")

    Stored as:
        user:name = "Imon"
        user:country = "Bangladesh"
        user:degree = "BSc CSE"
    """

    # Store all passed keyword arguments
    for key, value in kwargs.items():
        namespaced_key = f"user:{key}"
        tool_context.state[namespaced_key] = value

    return {"status": "success", "saved_fields": list(kwargs.keys())}


def retrieve_userinfo(tool_context: ToolContext) -> dict[str, any]:
    """
    Flexible tool to retrieve ALL user-related info 
    (keys starting with 'user:') from session state.
    Returns a clean dictionary without the prefix.
    """

    user_data = {}

    # Access the State object's combined dictionary safely
    current_state = {**tool_context.state._value, **tool_context.state._delta}

    for key, value in current_state.items():
        if key.startswith("user:"):
            clean_key = key.replace("user:", "")
            user_data[clean_key] = value

    return {"status": "success", "data": user_data}

print("✅ Tools created.")



# Wrapper functions for Dataset
dataset_path = "/kaggle/input/scholarships-clean/scholarships_clean.json"

def agent_scholarship_finder(profile: dict, top_k: int = 5) -> dict:
    """Finds scholarships for a user profile using a predefined dataset."""
    return find_scholarships(profile=profile, dataset_path=dataset_path, top_k=top_k)



InMemoryMemoryService - Built-in service for prototyping and testing (keyword matching, no persistence)
session_service → Manages conversation threads and events
memory_service → Provides long-term knowledge storage

using-load_memory (Reactive)
where -Agent decides when to search memory
Only retrieves when the agent thinks it's needed
More efficient (saves tokens)
Risk: Agent might forget to search

In [ ]:
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Use google_search tool to find scholarships worldwide.
    Return raw search results.
    """,
    tools=[google_search]
)


In [ ]:

scholarship_agent = LlmAgent(
    name="scholarship_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a smart scholarship recommendation assistant.

--- Memory Rules ---
1. Whenever the user explicitly provides their name or country, you MUST call the `save_userinfo` tool to store it in session state using the `user:` prefix.
2. Before giving any scholarship recommendations, ALWAYS call the `retrieve_userinfo` tool to check whether user information is already stored.
3. If the user's name or country is missing after retrieval, politely ask the user for the missing details.

--- Scholarship Search Rules ---
4. When the user provides a scholarship query (degree, preferred country, funding preference), first call the `agent_scholarship_finder` tool.
5. If the tool returns an empty list, you MUST call the `google_search_scholarships` tool as a fallback to find external scholarship information.
6. If both searches return no results, politely inform the user.

--- Output Formatting Rules ---
7. Format the final scholarship results as a bulleted list.
8. For EACH scholarship, use EXACTLY this template:

   **[Title]**
   * **Degrees**: [List of degrees]
   * **Funds**: [Funding amount]
   * **Country**: [Country]
   * **Deadline**: [Deadline]

9. If any information is missing, write "Not specified".
10. After the list, briefly explain WHY these scholarships match the user’s profile.""",
    tools=[agent_scholarship_finder,save_userinfo, retrieve_userinfo, AgentTool(agent=google_search_agent)] 
)

print("✅ Scholarship Finder Agent created with structured instructions.")
print("🔧 Available tool: agent_scholarship_finder")


# Define the local database URL
db_url = "sqlite:///scholarship_agent_data.db" 

# Initialize the DatabaseSessionService
session_service = DatabaseSessionService(db_url=db_url)

# --- 3. Step 2: Initialize the Persistent Runner ---
memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

# Use the base Runner class and pass the session_service argument
scholarship_runner = Runner(
    agent=scholarship_agent, 
    app_name=APP_NAME, 
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: scholarship_agent_data.db")
print(f"   - Sessions will survive restarts!")

await run_session(
    scholarship_runner,
    [
        "Hi, I am Sam! ",
        "I want to study in Any Country and I am looking for Any Funding Amount",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)


In [ ]:
#checking agent is stateful or not
await run_session(
    scholarship_runner,
    ["What did I ask you about earlier?", "And remind me, what's my name?"],
    "stateful-agentic-session",
)

In [ ]:
await run_session(
    scholarship_runner,
    ["What is the capital of India?", "Hello! What is my name?"],
    "test-db-session-01",
)

here i.e., two sessions do not share information


seeing whats in the database


In [ ]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("scholarship_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

In [ ]:
 
#events can become very large, leading to slower performance and higher costs.
from google.adk.apps.app import App, EventsCompactionConfig,ResumabilityConfig

scholarship_app_compacting = App(
    name="scholarship_app_compacting",  # Use the same app name as before
    root_agent=scholarship_agent,
    resumability_config=ResumabilityConfig(is_resumable=False),
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1, 
    ),
)

db_url = "sqlite:///scholarship_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)

# 3. Create the new Runner using the compacting App
scholarship_runner_compacting = Runner(
    app=scholarship_app_compacting, 
    session_service=session_service
)

print("✅ Scholarship App redefined with Events Compaction!")
print(f"🔧 Compaction Interval: {scholarship_app_compacting.events_compaction_config.compaction_interval} turns")
print("🚀 Use 'scholarship_runner_compacting' for optimized, persistent sessions.")



# 📝 Issue & Solution Note: *Resumability + Compaction Error*

## ❗ Issue

When using **Google ADK agents**, enabling **`resumability=True`** and **`compaction=True`** at the same time causes session metadata corruption.
This results in runtime errors such as:

```
'dict' object has no attribute 'start_timestamp'
```

### **Why it happens**

* `resumability` stores metadata (e.g., timestamps, history pointers).
* `compaction` rewrites/compacts session data.
* These two mechanisms **conflict internally**, leading to malformed metadata stored in the database.

---

## ✅ Solution

### **1. Disable resumability when compaction is used**

```python
session_service = DatabaseSessionService(
    database=...,
    resumable=False
)
```

### **2. Delete the old database/session**

If the data is already corrupted, you must **remove/reset** the previous session storage (local DB or Firestore).

### **3. Re-run the agent**

After resetting the DB and disabling `resumability`, the agent will run correctly without errors.

---

## ✔ Final Recommendation

> **Never use `resumability=True` and `compaction=True` together.**
> For projects using compaction, always set `resumable=False`.

In [ ]:
#turn1
await run_session(
    scholarship_runner_compacting,
    "my name is imon from Bangladesh.I am a Masters student in the USA. Find me scholarships with $1000 funding .",
    "Compaction_demo",
)
# --- Turn 2:
await run_session(
    scholarship_runner_compacting,
    "Based on those results, can you tell me the deadline for the first scholarship you found?",
    "Compaction_demo",
)
# --- Turn 3:
await run_session(
    scholarship_runner_compacting,
    "Which of those scholarships is also open to Bachelor students?",
    "Compaction_demo",
)
#turn 4
await run_session(
    scholarship_runner_compacting,
    "What country did I say I was interested in for my studies?",
    "Compaction_demo",
)

In [ ]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=scholarship_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="Compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

**Here we create Cv/sop agent with HITL pause and resume as we see it in the 
course materials 2(b) ,**

In [ ]:
import uuid
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.function_tool import FunctionTool

# 1. Define the Review Tool 
def submit_draft_for_review(
    draft_text: str, document_type: str, tool_context: ToolContext
) -> dict:
    """
    Submits a draft (SOP or CV) for human review.
    This tool PAUSES execution until the human approves or rejects.
    """
    
    # SCENARIO 1: First call - PAUSE for Human Review
    if not tool_context.tool_confirmation:
        
        # We attach the FULL DRAFT to the confirmation payload so the human can see it
        tool_context.request_confirmation(
            hint=f"Review {document_type} Draft",
            payload={
                "document_type": document_type,
                "draft_text": draft_text 
            },
        )
        
        return { 
            "status": "pending_review",
            "message": f"Draft submitted. Waiting for human feedback on: {document_type}"
        }

    # SCENARIO 2: Resuming after Human Decision
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "message": "The human APPROVED the draft. You can now finalize formatting and output the result."
        }
    else:
        return {
            "status": "rejected",
            "message": "The human REJECTED the draft. Please ask the user for specific feedback on what to change."
        }


print("✅ Review Tool created.")

In [ ]:
refiner_agent = LlmAgent(
    name="refiner_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are an academic editor.
    Your job is to take raw text and return:
    - grammatically correct
    - more concise
    - academic tone
    - logically structured
    Do NOT add new ideas. Only improve writing.
    """,
)


In [ ]:
# 1️⃣ Redefine SOP Agent as TEXT ONLY (No change needed, this is robust)
sop_agent = LlmAgent(
    name="sop_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are an expert academic SOP writer.
    1. Only generate a professional SOP based on the user profile and scholarship details.
    2. Do NOT call any tools.
    3. Do NOT save anything. Just output the text of the SOP.
    """
)

# 2️⃣ Redefine CV Agent as TEXT ONLY (No change needed, this is robust)
cv_agent = LlmAgent(
    name="cv_agent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are an expert CV writer.
    1. Only generate a professional CV based on the user profile and scholarship details.
    2. Do NOT call any tools.
    3. Do NOT save anything. Just output the text of the CV.
    """
)

print("✅ SOP and CV Agents")


In [ ]:
from google.adk.apps.app import App, ResumabilityConfig

# 3️⃣ Orchestrator Agent: handles all tool calls including HITL
orchestrator_agent = LlmAgent(
    name="orchestrator_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Orchestrator for the Scholarship System.
    
    1. At the start of a new session, ALWAYS call `retrieve_userinfo` to check for saved user data.
    2. If the user asks to find scholarships, call the `scholarship_agent`.
    
    3. **Document Generation Pipeline (SOP/CV):** When the user asks to generate a CV or SOP, you MUST follow this strict sequence:
        a. **Generate Draft:** Call the appropriate sub-agent (`cv_agent` or `sop_agent`) to generate the draft text.
        b. **Refine Draft (AITL):** Immediately call the `refiner_agent` tool, passing the generated draft text as input. The refiner will return the polished draft.
        c. **DISPLAY DRAFT TO USER:** Immediately output the polished draft text (the result of the refiner_agent) to the user.
        d. **Submit for HITL:** Then, immediately call the `submit_draft_for_review` tool, passing the SAME polished draft text to the tool.
        e. Wait for the tool's status (approval or rejection) and inform the user of the outcome, asking them to confirm the submission.
    4. Once the draft is approved by the human, call `save_userinfo` to store the final output (e.g., user:last_sop or user:last_cv).
    5. Always return concise, clear, and action-oriented responses to the user.
    """,
    tools=[
        save_userinfo,
        retrieve_userinfo,
        # API FIX: Removed 'name' argument from AgentTool
        AgentTool(agent=scholarship_agent), 
        AgentTool(agent=sop_agent),
        AgentTool(agent=cv_agent),
        FunctionTool(func=submit_draft_for_review),
        AgentTool(agent=refiner_agent),
        AgentTool(agent=google_search_agent),
    ],
)

# 4️⃣ Set up database-backed session (No change needed)
db_url = "sqlite:///scholarship_orchestrator.db"
session_service = DatabaseSessionService(db_url=db_url)

# 5️⃣ Wrap Orchestrator in a resumable App (Assuming necessary imports are present)
orchestrator_app = App(
    name="scholarship_orchestrator_app",
    root_agent=orchestrator_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)


print("✅ Orchestrator App ready with robust configuration.")



# 📝 Why `resumability=True` Is Used (And Why It’s Safe Here)

## ✅ Reason for Using `resumability=True`

In the **Orchestrator Agent**, `resumability=True` is intentionally enabled because:

### ✔ The Orchestrator needs long-term memory

It must **remember previous user data**, such as:

* last generated CV/SOP
* stored scholarship preferences
* previous HITL review status
* user profile information

This memory is required for:

* calling `retrieve_userinfo` at the start of each new session
* tracking document creation workflows
* maintaining continuity between conversations

So **resumability is essential** for the Orchestrator.

---

## 🚫 Why This Does NOT cause errors

You **did NOT enable Events Compaction** in this setup.

### ❗ Important Rule

> **Resumability and Compaction cannot be used together.**

Since you **only enabled resumability**, and **no compaction**, the session metadata is stored correctly.
Therefore, no corruption occurs, and no `start_timestamp` error will appear.


In [ ]:

# Initialize the Runner using the 'app' parameter
orchestrator_runner = Runner(
    app=orchestrator_app,            
    session_service=session_service, 
    memory_service=memory_service,  
)

print("✅ Orchestrator Runner initialized successfully!")

In [ ]:
await run_session(
    orchestrator_runner,
    [
       "Hi, I am Imon! ",
        "I want to study in USA for a Master's in Data Science and I am looking for a Funding Amount $1000",
        "Find some scholarships"
    ],
    "orchestrator-test-session-01"  # MUST be the same ID to maintain context
)

In [ ]:
# Ensure your helper function 'run_session' is defined (from your previous cells)
# If not, paste the helper function definition here first.

await run_session(
    orchestrator_runner,
    [
       " I am applying for a Master's in Data Science.",
        "The university is Stanford.", 
        "Can you help me write a Statement of Purpose (SOP) for it?",
        # 🚨 FIX: This 'Y' simulates the human reviewer approving the draft,
            # which prevents the request_confirmation() call from blocking the test runner indefinitely.
            "Y"
    ],
    "orchestrator-test-session-01"
)

In [ ]:
await run_session(
    orchestrator_runner,
    ["No further comments. Please refine and finalize the SOP."],
    "orchestrator-test-session-01"
)


In [ ]:
await run_session(
    orchestrator_runner,
    [
        "I, as the human reviewer, approve the draft. Please finalize the SOP."

    ],
    "orchestrator-test-session-01" # Continue the session
)

In [ ]:
await run_session(
    orchestrator_runner,
    ["Show me the finalized SOP."],
    "orchestrator-test-session-01"
)


In [ ]:
await run_session(
    orchestrator_runner,
    [
        "As the user, I request you to output the finalized SOP text again. "
        "You generated and refined it earlier. Please reproduce the finalized SOP content directly now."
    ],
    "orchestrator-test-session-01"
)


In [ ]:
# Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("scholarship_agent_data.db"):
    os.remove("scholarship_agent_data.db")
print("✅ Cleaned up old database files")